# ------ **Import Subroutines and Settings** -------

# *Procedure* for building " *The df_ib DataFrame from ib770 & ib970 combined with meta_data*

In [1]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")

/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


In [2]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
from collections.abc import Mapping
import re
#from sys_funcs import 

In [430]:
# set print rows  This worksheet sets maximum # of rows printed
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows to display
# pd.reset_option('display.max_rows')  
print('print set to 1000 rows max' )

print set to 1000 rows max


In [432]:
print("NOTE: timestamp = Test Date / Time does not work  use computed time stamp")

NOTE: timestamp = Test Date / Time does not work  use computed time stamp


# Pre-concatinate processing import and prepare 
1. **"df_m_ib_tst"**
2. **"df_m_ib_tst_nms"**
3. **"df_ib97_tst"**
4. **"df_ib97_tst_nms"**
5. **"df_ib77_tst"**
6. **"df_ib77_tst_nms"**
7. **"_______________"**


## def functions called in data importing & refinment.

In [433]:
# def strip_numbers_from_columns(df):
import re

def strip_numbers_from_columns(df):
    """
    Removes leading or trailing numbers (and optional separators)
    from column names without altering descriptor text.
    Examples cleaned:
      '1 Weight' → 'Weight'
      'Weight 1' → 'Weight'
      '2-Height' → 'Height'
      '3_TBW' → 'TBW'
      '4:LBM' → 'LBM'
    """
    df = df.copy()
    new_cols = {}

    for col in df.columns:
        # Remove leading numbers + separators
        cleaned = re.sub(r'^\d+[\s\-\_:]*', '', col)
        # Remove trailing numbers + separators
        cleaned = re.sub(r'[\s\-\_:]*\d+$', '', cleaned)
        new_cols[col] = cleaned

    df = df.rename(columns=new_cols)
    return df


In [257]:
#filling the media cols
def fill_ib_leading_columns(df):
    """
    Fills the 5 leading operator columns for InBody datasets:
      - timestamp  ← parsed from 'Test Date / Time' (YYYYMMDDHHMMSS)
      - dtv        ← days since 1900‑01‑01
      - ib_id      ← 'mrn' if test time 03:00–23:59, else 'eve'
      - cls        ← NaN
      - cmmnts     ← NaN
    """

    df = df.copy()

    # --- 1. timestamp (correct parsing) ------------------------
    df['timestamp'] = pd.to_datetime(
        df['Test Date / Time'].astype(str),
        format="%Y%m%d%H%M%S",
        errors="coerce"
    )

    # --- 2. dtv: days since 1900‑01‑01 -------------------------
    origin = pd.Timestamp("1900-01-01")
    df['dtv'] = (df['timestamp'] - origin).dt.days

    # --- 3. ib_id classification -------------------------------
    def classify_ib_id(ts):
        if pd.isna(ts):
            return np.nan
        hour = ts.hour
        return "mrn" if 3 <= hour <= 12 else "eve"

    df['ib_id'] = df['timestamp'].apply(classify_ib_id)

    # --- 4. cls ------------------------------------------------
    df['cls'] = np.nan

    # --- 5. cmmnts ---------------------------------------------
    df['cmmnts'] = np.nan

    return df


In [258]:
# Sort the rows by timestamp
def sort_by_timestamp(df):
    """
    Sorts an InBody dataframe by the 'timestamp' column
    in ascending chronological order.
    """
    df = df.copy()
    df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    return df


In [175]:
# remove duplicates on the basis of timestamp
def remove_ib_duplicates(df, subset_cols=None):
    """
    Removes duplicate InBody rows based on key identifying columns.
    Default behavior: remove duplicates based on ['ID', 'timestamp'].
    """
    df = df.copy()

    # Default duplicate definition
    if subset_cols is None:
        subset_cols = ['timestamp']
        # subset_cols = ['ID', 'timestamp']
    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=subset_cols, keep='first')

    # Reset index for cleanliness
    df = df.reset_index(drop=True)

    return df


## Read latest **[m_ib_tst]** from xl and make a data frame **"df_m_ib_tst"** and list of cols **"df_m_ib_tst_nms"**

### Bringing in the main data files from Excel

In [457]:
#this brings in themain data file from Excel
df_m_ib_tst = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/m_ib_tst.csv")
df_m_ib_tst_nn = strip_numbers_from_columns(df_m_ib_tst)
# verify df_m_ib_tst_nn

In [458]:
# df_m_ib_tst_nn slicing
#verify print(df_m_ib_tst_nn["ECW/TBW"])  # list form wo meta

# ADD MEDA , CLEAN AND SORT  >>>>   **df_m_ib_tst_nn**

## Fill in the meta columns by reading the data in the rest of the columns >>> **df_m_ib_tst_nn**

In [460]:
# fill in meta columns
df_m_ib_tst_nn = fill_ib_leading_columns(df_m_ib_tst_nn)
print("meta cols filled")
# verify df_m_ib_tst_nn

meta cols filled


#### Test the **"media_lst"** calculations of the timestamp, DTV, ib_id , cls , cmmnts

In [461]:
media_lst = [
    "Test Date / Time",
    "timestamp",
    "dtv",
    "ib_id",
    "cls",
    "cmmnts"
]


In [464]:
# verify df_m_ib_tst_nn[media_lst].head(20)

## Sort the data frame on the basis of timestamp you >> **df_m_ib_tst_s**

In [465]:
df_m_ib_tst_nn_s = sort_by_timestamp(df_m_ib_tst_nn)

In [466]:
# verify 
df_m_ib_tst_nn_s[media_lst].head(20)

,Test Date / Time,timestamp,dtv,ib_id,cls,cmmnts
0,20250922225826,2025-09-22 22:58:26,45920,eve,NaN,NaN
1,20251211092610,2025-12-11 09:26:10,46000,mrn,NaN,NaN
2,20251211092610,2025-12-11 09:26:10,46000,mrn,NaN,NaN
3,20251211092610,2025-12-11 09:26:10,46000,mrn,NaN,NaN
4,20251212084231,2025-12-12 08:42:31,46001,mrn,NaN,NaN
5,20251212084231,2025-12-12 08:42:31,46001,mrn,NaN,NaN
6,20251212084231,2025-12-12 08:42:31,46001,mrn,NaN,NaN
7,20251212221713,2025-12-12 22:17:13,46001,eve,NaN,NaN
8,20251212221713,2025-12-12 22:17:13,46001,eve,NaN,NaN
9,20251212221713,2025-12-12 22:17:13,46001,eve,NaN,NaN


In [468]:
#verify df_m_ib_tst_nn_s[["dtv", "ib_id" , "SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

## Remove duplicates [...xd]  Retaining the first of a duplicate pair  >>> **df_m_ib_tst_s_xd**

In [470]:
df_m_ib_tst_nn_s_xd  =  remove_ib_duplicates(df_m_ib_tst_s, subset_cols=None)
# verify df_m_ib_tst_nn_s_xd

## Slicing the fully processed data frame [df_m_ib_tst_nn_s_xd] to display columns of interest  

# working cell

In [476]:
#verify 
df_m_ib_tst_nn_s_xd[["dtv","timestamp","SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

,dtv,timestamp,SMM (Skeletal Muscle Mass),Weight,BMR (Basal Metabolic Rate),ECW/TBW
0,45920,2025-09-22 22:58:26,98.3,262.4,2151,0.413
1,46000,2025-12-11 09:26:10,99.4,260.1,2158,0.410
2,46001,2025-12-12 08:42:31,101.2,259.2,2186,0.409
3,46001,2025-12-12 22:17:13,101.6,262.3,2208,0.415
4,46002,2025-12-13 05:49:21,102.3,260.0,2203,0.410
5,46003,2025-12-14 09:03:36,99.2,258.0,2148,0.407
6,46003,2025-12-14 23:48:14,99.2,261.1,2160,0.414
7,46004,2025-12-15 08:00:38,99.9,258.3,2155,0.406
8,46004,2025-12-15 23:33:24,103.4,260.0,2230,0.413
9,46005,2025-12-16 06:48:08,98.3,257.0,2129,0.406


# The stored data is loaded checked and verified, and ready to receive new data
1. **"df_m_ib_tst"**
2. **"df_m_ib_tst_nms"**
3. **"df_ib97_tst"**
4. **"df_ib97_tst_nms"**
5. **"df_ib77_tst"**
6. **"df_ib77_tst_nms"**
7. **"_______________"**

In [327]:
df_m_ib_tst_nms = list(df_m_ib_tst.columns)
#df_ib77_raw_nms with numbers 
#verify  df_m_ib_tst_nms
print("df_m_ib_tst_nms w/o numbers OK")

df_m_ib_tst_nms w/o numbers OK


In [329]:
# Testing duplicates in the column titles
df_m_ib_tst.columns[df_m_ib_tst.columns.duplicated()]
print("df_m_ib_tst_nms w/o numbers no duplicate OK")

df_m_ib_tst_nms w/o numbers no duplicate OK


## Read the latest data from 770 and 970 in the data folder of repo

### ib77:  This segment reads the data from the Excel file_ib77 and computes the names of the column heads, strips them of numbers, puts them in a List, and records them in a pickle. *ib77_raw_nms*

In [477]:
# loads the new from the 770
df_ib77_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib77",
    pattern="*"
)
# verify 
df_ib77_raw
#print("df_ib77_raw w/o numbers OK")

✅ Loaded 091725_20251225081906.csv with utf-8
✅ Loaded 091725_20251213054921.csv with utf-8
✅ Loaded 091725_20251214234814.csv with utf-8
✅ Loaded 091725_20251223231059.csv with utf-8
✅ Loaded 091725_20251217075000.csv with utf-8
✅ Loaded 091725_20251218081604.csv with utf-8
✅ Loaded 091725_20251221071026.csv with utf-8
✅ Loaded 091725_20251216233809.csv with utf-8
✅ Loaded 091725_20251215080038.csv with utf-8
✅ Loaded 091725-1_20251212221713.csv with utf-8
✅ Loaded 091725_20251219080729.csv with utf-8
✅ Loaded 091725_20251221205127.csv with utf-8
✅ Loaded 091725_20251226083412.csv with utf-8
✅ Loaded 091725_20251227083952.csv with utf-8
✅ Loaded 091725_20251214090336.csv with utf-8
✅ Loaded 091725_20251218212747.csv with utf-8
✅ Loaded 091725_20251224080342.csv with utf-8
✅ Loaded 091725_20251222221501.csv with utf-8
✅ Loaded 091725_20251219232758.csv with utf-8
✅ Loaded 091725_20251215233324.csv with utf-8
✅ Loaded 091725_20251224234318.csv with utf-8
✅ Loaded 091725_20251216064808.c

,1. Name,2. ID,3. Height,4. Date of Birth,5. Age,6. Mobile Number,7. Phone Number,8. Zip Code,9. Address,10. E-mail,...,143. R/Ht,144. Xc/Ht,145. HGS of Left Arm 1st,146. HGS of Left Arm 2nd,147. HGS of Right Arm 1st,148. HGS of Right Arm 2nd,149. HGS/WT,Unnamed: 149,source_file,encoding_used
0,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251225081906.csv,utf-8
1,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
2,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
3,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251223231059.csv,utf-8
4,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251217075000.csv,utf-8
5,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251218081604.csv,utf-8
6,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251221071026.csv,utf-8
7,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8
8,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8
9,91725,6ft 06.0in,1939.02.23.,F,86.0,-,-,-,-,-,...,0.0,0.0,-,-,-,-,-,NaN,091725-1_20251212221713.csv,utf-8


In [504]:
df_ib77_nn = strip_numbers_from_columns(df_ib77_raw)
#verify 
print(list(df_ib77_nn.columns))

df_ib77_nn[['. ID','. Test Date / Time','. ECW/TBW']]  #,"SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

['. Name', '. ID', '. Height', '. Date of Birth', '. Age', '. Mobile Number', '. Phone Number', '. Zip Code', '. Address', '. E-mail', '. Date of Registration', '. Memo', '. Test Date / Time', '. Weight', '. TBW (Total Body Water)', '. ICW (Intracellular Water)', '. ECW (Extracellular Water)', '. DLM (Dry Lean Mass)', '. BFM (Body Fat Mass)', '. LBM (Lean Body Mass)', '. SMM (Skeletal Muscle Mass)', '. BMI (Body Mass Index)', '. PBF (Percent Body Fat)', '. LBM of Right Arm', '. LBM% of Right Arm', '. LBM of Left Arm', '. LBM% of Left Arm', '. LBM of Trunk', '. LBM% of Trunk', '. LBM of Right Leg', '. LBM% of Right Leg', '. LBM of Left Leg', '. LBM% of Left Leg', '. Leg Lean Mass', '. TBW of Right Arm', '. TBW of Left Arm', '. TBW of Trunk', '. TBW of Right Leg', '. TBW of Left Leg', '. ICW of Right Arm', '. ICW of Left Arm', '. ICW of Trunk', '. ICW of Right Leg', '. ICW of Left Leg', '. ECW of Right Arm', '. ECW of Left Arm', '. ECW of Trunk', '. ECW of Right Leg', '. ECW of Left Leg'

,. ID,. Test Date / Time,. ECW/TBW
0,6ft 06.0in,20251225081906,0.399
1,6ft 06.0in,20251213054921,0.410
2,6ft 06.0in,20251214234814,0.414
3,6ft 06.0in,20251223231059,0.405
4,6ft 06.0in,20251217075000,0.404
5,6ft 06.0in,20251218081604,0.403
6,6ft 06.0in,20251221071026,0.400
7,6ft 06.0in,20251216233809,0.411
8,6ft 06.0in,20251215080038,0.406
9,6ft 06.0in,20251212221713,0.415


In [479]:
df_ib77_raw_nms = list(df_ib77_raw.columns)
# VERIFY df_ib77_raw_nms
print ("df_ib77_raw_nms with numbers OK")

df_ib77_raw_nms with numbers OK


In [480]:
# Testing duplicates in the column titles
df_ib77_raw.columns[df_ib77_raw.columns.duplicated()]
print ("df_ib77_raw_nms with numbers no duplicates OK")

df_ib77_raw_nms with numbers no duplicates OK


In [482]:
# strip numbers off So that the identical names in ib77 are the same as 97
df_ib77_raw_nms_nn = strip_col_numbers(df_ib77_raw)
# Verify 
df_ib77_raw_nms_nn 
#print ("df_ib77_raw_nms_nn ie. w/o numbers  OK")

['Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'TBW (Total Body Water)',
 'ICW (Intracellular Water)',
 'ECW (Extracellular Water)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'LBM (Lean Body Mass)',
 'SMM (Skeletal Muscle Mass)',
 'BMI (Body Mass Index)',
 'PBF (Percent Body Fat)',
 'LBM of Right Arm',
 'LBM% of Right Arm',
 'LBM of Left Arm',
 'LBM% of Left Arm',
 'LBM of Trunk',
 'LBM% of Trunk',
 'LBM of Right Leg',
 'LBM% of Right Leg',
 'LBM of Left Leg',
 'LBM% of Left Leg',
 'Leg Lean Mass',
 'TBW of Right Arm',
 'TBW of Left Arm',
 'TBW of Trunk',
 'TBW of Right Leg',
 'TBW of Left Leg',
 'ICW of Right Arm',
 'ICW of Left Arm',
 'ICW of Trunk',
 'ICW of Right Leg',
 'ICW of Left Leg',
 'ECW of Right Arm',
 'ECW of Left Arm',
 'ECW of Trunk',
 'ECW of Right Leg',
 'ECW of Left Leg',
 'ECW/TBW',
 'ECW/TBW of Right Arm',
 'ECW/TBW o

In [483]:
# Testing duplicates in the column titles
df_ib77_raw.columns[df_ib77_raw.columns.duplicated()]
print ("w/o numbers no duplicates OK")

w/o numbers no duplicates OK


In [484]:
# Load the IB77 Columns without numbers to pickle 
with open("df_ib77_raw_nms.pkl", "wb") as f:
    pickle.dump(df_ib77_raw_nms, f)
print ("df_ib77_raw_nms.pkl OK")

df_ib77_raw_nms.pkl OK


In [485]:
# READ the IB77 Columns without numbers FROM pickle 
with open("df_ib77_raw_nms.pkl", "rb") as f:
    df_ib77_raw_nms = pickle.load(f)
# Verify df_ib77_raw_nms
print ("from pkl df_ib77_raw_nms.pkl OK")

from pkl df_ib77_raw_nms.pkl OK


#### Calculate excess columns in the main data frame compared the columns in the IB77 data frame.

In [486]:
# Verify that we have the main data frame
df_m_ib_tst.columns

Index(['dtv', 'timestamp', 'cls', 'ib_id', 'cmmnts', 'Name', 'ID', 'Height',
       'Date of Birth', 'Age',
       ...
       'Weight_Z score', '50kHz-Whole Body Phase Angle_T score',
       '50kHz-Whole Body Phase Angle_Z score', 'TBW/WT_T Score',
       'TBW/WT_Z Score', 'SMI(SMM/Wt)_T score', 'SMI(SMM/Wt)_Z score',
       'ECM/BCM_T Score', 'ECM/BCM Z Score', 'Unnamed: 250'],
      dtype='object', length=325)

In [487]:
#Verify that we have data from data frame to be added to the main data frame If it's not there read it from pickle
# verify df_ib77_raw.columns

# This will load the ib77 into the accumulated data

In [488]:
# verify df_m_ib_tst
print(" The main df_m_ib_tst is in place with col numbers:")

 The main df_m_ib_tst is in place with col numbers:


In [489]:
# verify df_m_ib_tst_nn
df_m_ib_tst_nn = strip_numbers_from_columns(df_m_ib_tst)
df_m_ib_tst_nn
print(" The main df_m_ib_tst_nn is in place w/o col numbers:")

 The main df_m_ib_tst_nn is in place w/o col numbers:


In [495]:
# This is the IB77 columns that still have the numbers attached
ib77_raw_missing_cols_n = [col for col in ib77_raw.columns 
                           if col not in df_m_ib_tst_nn.columns]

# verify ib77_raw_missing_cols

In [293]:
for col in ib77_raw_missing_cols:
    ib77_raw[col] = np.nan
ib77_raw              # ["ECW/TBW"]

,1. Name,2. ID,3. Height,4. Date of Birth,5. Age,6. Mobile Number,7. Phone Number,8. Zip Code,9. Address,10. E-mail,...,143. R/Ht,144. Xc/Ht,145. HGS of Left Arm 1st,146. HGS of Left Arm 2nd,147. HGS of Right Arm 1st,148. HGS of Right Arm 2nd,149. HGS/WT,Unnamed: 149,source_file,encoding_used
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725_20251225081906.csv,utf-8
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725_20251213054921.csv,utf-8
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725_20251214234814.csv,utf-8
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725_20251223231059.csv,utf-8
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725_20251217075000.csv,utf-8
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725_20251218081604.csv,utf-8
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725_20251221071026.csv,utf-8
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725_20251216233809.csv,utf-8
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725_20251215080038.csv,utf-8
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,091725-1_20251212221713.csv,utf-8


In [ ]:
import pandas as pd

# Suppose missing_cols is a list of column names you want to add
new_cols = pd.DataFrame({col: [None] * len(ib77_raw) for col in missing_cols})
new_cols

In [ ]:
# Add them in one operation
ib77_raw = pd.concat([ib77_raw, new_cols], axis=1)

In [ ]:
# Add any missing columns to IB77_raw
for col in df_m_ib_tst.columns:
    if col not in ib77_raw.columns:
        ib77_raw[col] = None
# verify 
ib77_raw

In [ ]:
df_ib77_raw[["BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

In [396]:
# The main data frame is larger than the add data frame so we need To determine the Missing columns.
missing_cols = [c for c in df_m_ib_tst.columns if c not in df_ib77_raw.columns]
#verify missing_cols

In [398]:
# Use the missing_cols dataframe in order add columns with NAn values to the ib77 values so it can be appended to with the main data frame "df_m_ib_tst
df_ib77_cncnt = df_ib77_raw.copy()    # start building the df
#verify df_ib77_cncnt

In [400]:
# 1. Add missing columns
for col in missing_cols:
    df_ib77_cncnt[col] = np.nan
#verify df_ib77_cncnt

In [404]:
# 3. Reorder to match master schema
df_ib77_cncnt = df_ib77_cncnt[df_m_ib_tst.columns]
#verify df_ib77_cncnt

In [407]:
# 4. Append
df_m_ib_tst = pd.concat([df_m_ib_tst, df_ib77_cncnt], ignore_index=True)
#verify df_m_ib_tst

/tmp/ipykernel_864131/1781342986.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_m_ib_tst = pd.concat([df_m_ib_tst, df_ib77_cncnt], ignore_index=True)


In [391]:
# Calculate missing columns when df_ib77 raw data is appended to IB_tst Data frame
df_ib77_raw_missing_cols_n = [col for col in df_ib77_raw.columns if col not in df_m_ib_tst.columns]
# verify 
df_ib77_raw_missing_cols_n
print ("df_ib77_raw_missing_cols_n with numbers  OK")

df_ib77_raw_missing_cols_n with numbers  OK


In [379]:
# strip numbers off So that the identical names in ib77 are the same as 97
df_ib77_raw_missing_cols_nms = strip_col_numbers(df_ib77_raw_missing_cols_n)
# Verify 
df_ib77_raw_missing_cols_nms
print ("df_ib77_raw_missing_cols_nms w/o numbers  OK")

df_ib77_raw_missing_cols_nms w/o numbers  OK


In [381]:
# Load the df_ib77_raw_missing_cols_nms to pickle 
with open("df_ib77_raw_missing_cols_nms.pkl", "wb") as f:
    pickle.dump(df_ib77_raw_missing_cols_nms, f)
print ("df_ib77_raw_missing_cols_nms.pkl OK")

df_ib77_raw_missing_cols_nms.pkl OK


In [376]:
# READ thedf_ib77_raw_missing_cols_nms FROM pickle 
with open("df_ib77_raw_missing_cols_nms.pkl", "rb") as f:
    df_ib77_raw_missing_cols_nms = pickle.load(f)
# Verify df_ib77_raw_missing_cols_nms
print ("from pkl df_ib77_raw_missing_cols_nms.pkl OK")

from pkl df_ib77_raw_missing_cols_nms.pkl OK


In [377]:
# Calculate missing columns when df_ib77 raw data is appended to ib_tst DataFrame

## Calculate the **ib_tst** columns that are not used when **ib77** is concatenated =  **xcss77**

### ib97:  This segment Reads the data from the excel file_ib97 and computes names of the column head, strips them of numbers and Puts them in a list and records them in a pickle. *ib97_raw_nms*

In [277]:
# loads the new from the 970
ib97_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97", 
    pattern="*"
)

✅ Loaded 251201-1_20251207201219.csv with ISO-8859-1
✅ Loaded 091725-1_20251118042051.csv with ISO-8859-1
✅ Loaded 091725-1_20251023065307.csv with ISO-8859-1
✅ Loaded 091725-1_20250928235640.csv with ISO-8859-1
✅ Loaded 091725-1_20251025233431.csv with ISO-8859-1
✅ Loaded 091725-1_20251124075021.csv with ISO-8859-1
✅ Loaded 091725-1_20251021000847.csv with ISO-8859-1
✅ Loaded 091725-1_20251101213555.csv with ISO-8859-1
✅ Loaded 251201-1_20251205081353.csv with ISO-8859-1
✅ Loaded 091725-1_20251116230333.csv with ISO-8859-1
✅ Loaded 251201-1_20251206072118.csv with ISO-8859-1
✅ Loaded 091725-1_20251130055027.csv with ISO-8859-1
✅ Loaded 091725-1_20251019230750.csv with ISO-8859-1
✅ Loaded 091725-1_20251017091340.csv with ISO-8859-1
✅ Loaded 091725-1_20251002065435.csv with ISO-8859-1
✅ Loaded 091725-1_20251019074606.csv with ISO-8859-1
✅ Loaded 091725-1_20251118230931.csv with ISO-8859-1
✅ Loaded 091725-1_20251108074302.csv with ISO-8859-1
✅ Loaded 091725-1_20251028225751.csv with ISO-

In [281]:
ib97_raw_nms = list(ib97_raw.columns)
# Verify 
ib97_raw_nms

['1. Name',
 '2. ID',
 '3. Height',
 '4. Date of Birth',
 '5. Age',
 '6. Mobile Number',
 '7. Phone Number',
 '8. Zip Code',
 '9. Address',
 '10. E-mail',
 '11. Date of Registration',
 '12. Memo',
 '13. Test Date / Time',
 '14. Weight',
 '15. Lower Limit (Weight Normal Range)',
 '16. Upper Limit (Weight Normal Range)',
 '17. TBW (Total Body Water)',
 '18. Lower Limit (TBW Normal Range)',
 '19. Upper Limit (TBW Normal Range)',
 '20. ICW (Intracellular Water)',
 '21. Lower Limit (ICW Normal Range)',
 '22. Upper Limit (ICW Normal Range)',
 '23. ECW (Extracellular Water)',
 '24. Lower Limit (ECW Normal Range)',
 '25. Upper Limit (ECW Normal Range)',
 '26. Protein',
 '27. Lower Limit (Protein Normal Range)',
 '28. Upper Limit (Protein Normal Range)',
 '29. Minerals',
 '30. Lower Limit (Minerals Normal Range)',
 '31. Upper Limit (Minerals Normal Range)',
 '32. DLM (Dry Lean Mass)',
 '33. BFM (Body Fat Mass)',
 '34. Lower Limit (BFM Normal Range)',
 '35. Upper Limit (BFM Normal Range)',
 '36.

In [282]:
# strip numbers off So that the identical names in ib97 are the same as ib97
ib97_raw_nms = strip_col_numbers(ib97_raw)
# Verify
ib97_raw_nms

['Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'Lower Limit (Weight Normal Range)',
 'Upper Limit (Weight Normal Range)',
 'TBW (Total Body Water)',
 'Lower Limit (TBW Normal Range)',
 'Upper Limit (TBW Normal Range)',
 'ICW (Intracellular Water)',
 'Lower Limit (ICW Normal Range)',
 'Upper Limit (ICW Normal Range)',
 'ECW (Extracellular Water)',
 'Lower Limit (ECW Normal Range)',
 'Upper Limit (ECW Normal Range)',
 'Protein',
 'Lower Limit (Protein Normal Range)',
 'Upper Limit (Protein Normal Range)',
 'Minerals',
 'Lower Limit (Minerals Normal Range)',
 'Upper Limit (Minerals Normal Range)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'Lower Limit (BFM Normal Range)',
 'Upper Limit (BFM Normal Range)',
 'FFM (Fat Free Mass)',
 'SMM (Skeletal Muscle Mass)',
 'Lower Limit (SMM Normal Range)',
 'Upper Limit (SMM Normal Range)',
 'BMI (Body

In [158]:
# Load the IB97 Columns without numbers to pickle 
with open("ib97_raw_nms.pkl", "wb") as f:
    pickle.dump(ib97_raw_nms, f)


In [162]:
# READ the IB97 Columns without numbers to pickle 
with open("ib97_raw_nms.pkl", "rb") as f:
    ib97_raw_nms = pickle.load(f)
# Verify ib97_raw_nms

#### The following data are loaded and ready to be consolidated into an updated Total data data frame "m_bi_tst" from :XL >>> "df_m_bi_tst" and new"df_ib97_raw",new "df_ib77_raw"

### This will load the ib97 into the accumulated data

In [ ]:
# Add any missing columns to IB97_raw
for col in m_ib_tst.columns:
    if col not in ib97_raw.columns:
        ib97_raw[col] = None
# Reorder IB97_raw columns to match m_ib_tst
ib97_raw = ib97_raw[m_ib_tst.columns]


In [ ]:
m_ib_tst = pd.concat([m_ib_tst, ib97_raw], ignore_index=True)
m_ib_tst

In [43]:
# Reorder ib77_raw columns to match df_m_ib_tst
ib77_raw = ib77_raw[df_m_ib_tst.columns]

In [410]:
# This concatenates 770 new data with the existing data That contains the accumulator data from 770 and 970
df_m_ib_tst = pd.concat([df_m_ib_tst, ib77_raw], ignore_index=True)
#verify df_m_ib_tst

In [412]:
# This checks to see if column names are still all unique after the new data
df_m_ib_tst.columns[df_m_ib_tst.columns.duplicated()]
print(" No duplicates found Afternoon data was appended")

 No duplicates found Afternoon data was appended


In [ ]:
df_m_ib_tst["

In [93]:
ib77_raw.columns[ib77_raw.columns.duplicated()]


Index([], dtype='object')

In [413]:
df_m_ib_tst_s_xd[["dtv","SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

,dtv,SMM (Skeletal Muscle Mass),Weight,BMR (Basal Metabolic Rate),ECW/TBW
0,45920,98.3,262.4,2151,0.413
1,46000,99.4,260.1,2158,0.410
2,46001,101.2,259.2,2186,0.409
3,46001,101.6,262.3,2208,0.415
4,46002,102.3,260.0,2203,0.410
5,46003,99.2,258.0,2148,0.407
6,46003,99.2,261.1,2160,0.414
7,46004,99.9,258.3,2155,0.406
8,46004,103.4,260.0,2230,0.413
9,46005,98.3,257.0,2129,0.406


# load latest [m_ib_tst] to xl

In [ ]:
# load latest [m_ib_tst] from_xl
folder_path = "/home/bhuns/JL_2/data/ib_tst/m_ib_tst.csv"
m_ib_tst.to_csv(folder_path, index=False)
# verify
#
m_ib_tst

# Everything below this line is residual the days the template was developed and completed. Erase it when everything using the template is completed
# ================================================================================================================================================================

# Startup only Create  **df_ib_fls_tmplt**  ie  *[COL_NMS = meta+ib770dat + ib970dat]*

### Both the 77 and the 90 test datasets are convrted to dfs In order to determine *df_ib_tst_nms* Which is a single column data frame that all of the column names of the *df_ib_tst*

#### ✅ Concatenate → Preserve Order → Drop Duplicates

In [28]:
####  Concatenate → Preserve Order → Drop Duplicates mmmmmmmmmnnnnnnnllllll
# Concatenate in the required order
df_ib_tst_nms = pd.concat(
    [df_ib77_raw_nms, df_ib97_raw_nms],
    axis=0,
    ignore_index=True
)
# print(df_ib_tst_nms)
# Remove duplicate rows, keeping the first occurrence (from df_ib77_raw_nms)
df_ib_tst_nms = df_ib_tst_nms.drop_duplicates(keep="first")
# verify 
# print(df_ib_tst_nms)

In [ ]:
# Verifies that the duplicates have been removed and that the list has been compacted and there are not duplicates of 770 and 970 Col
df_ib_tst_nms[df_ib_tst_nms.duplicated()]

### *df_ib_tst_nms* must be converted to a simple list in order to made equal to the column heads of df_tst

In [ ]:
# The large test data frame that will be used to accept values of each from both 770 N 970 will be built on the basis of the df_ib_tst_nms, but this data bugs be in form of a strings.
# df['colname'].astype(str).tolist       # sign This is the sample given from copilot that is modified below for my situation 

ib_tst_lst = df_ib_tst_nms[0].astype(str).tolist()
# verify 
# print(ib_tst_lst)

In [ ]:
# this makes the template for the ib_tst data For both the 770 and the 970 It is made up of a 770 and a 970 with duplicates eliminated in a list of strings

df_ib_tst = pd.DataFrame(
    data = [[""] * len(df_ib_tst_lst)] * 2,
    columns = ib_tst_lst
)
#df_ib_tst_nms
df_ib_tst

In [ ]:
# Tests 
df_ib_tst["ID"]


In [ ]:
df_ib_tst.columns


In [ ]:
df_ib_tst["ID"]

## data frame of the main data storage_“ib_tst"

###  Make the edits to the structure of the main data spreadsheet that represents structure

### Then the CSV file into the worksheet to display the changes in Jupiter lab

In [ ]:
df_loaded = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/ib_tst.csv")
df_loaded.columns

In [ ]:
folder_path = "/home/bhuns/JL_2/data/ib_tst/ib_tst.csv"
df_ib_tst.to_csv(folder_path, index=False)
df_ib_tst

In [ ]:
df_loaded["Age"]



In [ ]:
#==================================================================================================================== Oh

###


# Align the columns via index suggestions from Copilot to insert new data into the major data frame

Put the creation of a data frame into its own worksheet

# ==============================================================================================================================================

In [429]:
# def strip_numbers_from_columns(df):
import re
def strip_numbers_from_columns(df):
    """
    Removes leading or trailing numbers (and optional separators)
    from column names without altering descriptor text.
    Examples cleaned:
      '1 Weight' → 'Weight'
      'Weight 1' → 'Weight'
      '2-Height' → 'Height'
      '3_TBW' → 'TBW'
      '4:LBM' → 'LBM'
    """
    df = df.copy()
    new_cols = {}

    for col in df.columns:
        # Remove leading numbers + separators
        cleaned = re.sub(r'^\d+[\s\-\_:]*', '', col)
        # Remove trailing numbers + separators
        cleaned = re.sub(r'[\s\-\_:]*\d+$', '', cleaned)
        new_cols[col] = cleaned

    df = df.rename(columns=new_cols)
    return df
